# --- Day 10: Pipe Maze ---
https://adventofcode.com/2023/day/10

In [3]:
def getPipes():
    with open("pipes.txt") as file:
        return file.read()

In [5]:
#Formatting
pipes = getPipes()
pipes = pipes.split("\n")
pipes = [list(x) for x in pipes]

#Find 'S' as the starting location:
for i, row in enumerate(pipes):
    for j, char in enumerate(row):
        if char == "S":
            currentCoords = [i, j] #Keeps track of current coordinates
            break

#Find the next move based on the location of the start ("S")
#Check for valid path up (x coordinate - 1)
if pipes[currentCoords[0] - 1][currentCoords[1]] in ['|', 'F', '7']:
    currentCoords[0] -= 1
    previousDirection = "Up"
#Check for valid path down (x coordinate - 1) if there is no valid path up
elif pipes[currentCoords[0] + 1][currentCoords[1]] in ['|', 'L', 'J']:
    currentCoords[0] += 1
    previousDirection = "Down"
#The start has to be connected by exactly two sides, so if it's not up or down, it has to be left and right
else:
    pipes[1] -= 1
    previousDirection = "Left"
    

#Keep track of the number of steps taken
numSteps = 1

#Loop through until we get back to "S"
while pipes[currentCoords[0]][currentCoords[1]] != "S":
    
    #Keep track of the current character
    currentChar = pipes[currentCoords[0]][currentCoords[1]]
    
    if currentChar == "|":
        #If the previous direction was up, that means it came from the bottom, so we need to move up
        if previousDirection == "Up":
            currentCoords[0] -= 1
            previousDirection = "Up"
        #This means if came from above, so we have to move down
        else:
            currentCoords[0] += 1
            previousDirection = "Down"
            
    elif currentChar == "-":
        #If we came from the left, that means we continue to move left
        if previousDirection == "Left":
            currentCoords[1] -= 1
            previousDirection = "Left"
        #This means if came from the right, so we have to keep moving to the right
        else:
            currentCoords[1] += 1
            previousDirection = "Right"
            
    elif currentChar == "L":
        #If coming from the left, move up
        if previousDirection == "Left":
            currentCoords[0] -= 1
            previousDirection = "Up"
        #If coming from up, move to the right
        else:
            currentCoords[1] += 1
            previousDirection = "Right"
            
    elif currentChar == "J":
        #If coming from right, move up
        if previousDirection == "Right":
            currentCoords[0] -= 1
            previousDirection = "Up"
        #If coming from up move to the left
        else:
            currentCoords[1] -= 1
            previousDirection = "Left"
            
    elif currentChar == "7":
        #If coming from the bottom, move left
        if previousDirection == "Up":
            currentCoords[1] -= 1
            previousDirection = "Left"
        #If coming from the right, move down
        else:
            currentCoords[0] += 1
            previousDirection = "Down"
            
    elif currentChar == "F":
        #If coming from the bottom, move right
        if previousDirection == "Up":
            currentCoords[1] += 1
            previousDirection = "Right"
        #If coming from right, move down
        else:
            currentCoords[0] += 1
            previousDirection = "Down"
    
    #Incriment numSteps by 1
    numSteps += 1
    
print(f"Number of steps to the farthest spot from the animal: {numSteps//2}")

Number of steps to the farthest spot from the animal: 6867


# --- Part Two ---

For this part, I used some help from the AOC subreddit, I had a very similar idea to this method that wouldn't have worked due to a very small issue that this method fixes
It can be found on this reddit thread:
https://www.reddit.com/r/adventofcode/comments/18f1sgh/2023_day_10_part_2_advise_on_part_2/

In [7]:
def spacePipes(pipes):
    """This function adds empty space between the pipes to make it so that we can recurse between pipes that 
    are not connected"""
    #Put spaces between characters so we can recurse between pipes
    #Puts filler spaces between rows
    spacedPipes = []
    for i, row in enumerate(pipes):
        spacedPipes.append([])

        #Calculate the length of the row to avoid extra filler spaces
        if len(row)*2 % 2 == 0:
            rowLength = (len(row) * 2) - 1
        else:
            rowLength = len(row) * 2

        for j in range(rowLength):
            if j % 2 == 0:
                spacedPipes[-1].append(pipes[i][j//2])
            else:
                if pipes[i][j//2] in ["-", "F", "L"] and pipes[i][(j//2) + 1] in ["-", "J", "7"]:
                    spacedPipes[-1].append("-")
                else:
                    spacedPipes[-1].append("#")

    tempSpacedPipes = spacedPipes
    spacedPipes = []
    for i in range(len(tempSpacedPipes) * 2):

        spacedPipes.append([])

        for j in range(len(tempSpacedPipes[i//2])):
            if i % 2 == 0:
                spacedPipes[-1].append(tempSpacedPipes[i//2][j])
            else:
                if tempSpacedPipes[i//2][j] in ["|", "F", "7"] and tempSpacedPipes[(i//2) + 1][j] in ["|", "J", "L"]:
                    spacedPipes[-1].append("|")
                else:
                    spacedPipes[-1].append("#")

    #Gets rid of additional row spacing at the end
    if len(set(spacedPipes[-1])) == 1:
        spacedPipes = spacedPipes[:-1]
    
    return spacedPipes

def fillBoard(pipes, startXCoordinate, startYCoordinate):
    """Fills the pipe map with points marked as outside points"""
    #Mark first point as an outside point
    pipes[startXCoordinate][startYCoordinate] = "O"
    #Keeps track of all coordinates that are outside points
    outsideCoords = [[startXCoordinate, startYCoordinate]]
    previousOutsideCoords = []
    
    #Keep looping until there is no change in the coordinates
    while outsideCoords != previousOutsideCoords:
        previousOutsideCoords = outsideCoords
        #Loop through all coordinates
        for coords in outsideCoords:
            #Checks to make sure the coordinate above the current coordinate is in bounds
            if isValidCoordinate(pipes, coords[0] - 1, coords[1]):
                #If the pipe is marked as either a filler space, or an inside point, mark it as an outside point
                if pipes[coords[0] - 1][coords[1]] in ["#", "I"] and [coords[0] - 1, coords[1]] not in outsideCoords:
                    #Mark as outside point
                    pipes[coords[0] - 1][coords[1]] = "O"
                    #Add coords to outsideCoords
                    outsideCoords.append([coords[0] - 1, coords[1]])

            #Checks to make sure the coordinate below the current coordinate is in bounds
            if isValidCoordinate(pipes, coords[0] + 1, coords[1]):
                #If the pipe is marked as either a filler space, or an inside point, mark it as an outside point
                if pipes[coords[0] + 1][coords[1]] in ["#", "I"] and [coords[0] + 1, coords[1]] not in outsideCoords:
                    #Mark as outside point
                    pipes[coords[0] + 1][coords[1]] = "O"
                    #Add coords to outsideCoords
                    outsideCoords.append([coords[0] + 1, coords[1]])

            #Checks to make sure the coordinate to the left of the current coordinate is in bounds
            if isValidCoordinate(pipes, coords[0], coords[1] - 1):
                #If the pipe is marked as either a filler space, or an inside point, mark it as an outside point
                if pipes[coords[0]][coords[1] - 1] in ["#", "I"]  and [coords[0], coords[1] - 1] not in outsideCoords:
                    #Mark as outside point
                    pipes[coords[0]][coords[1] - 1] = "O"
                    #Add coords to outsideCoords
                    outsideCoords.append([coords[0], coords[1] - 1])

            #Checks to make sure the coordinate to the right of the current coordinate is in bounds
            if isValidCoordinate(pipes, coords[0], coords[1] + 1):
                #If the pipe is marked as either a filler space, or an inside point, mark it as an outside point
                if pipes[coords[0]][coords[1] + 1] in ["#", "I"] and [coords[0], coords[1] + 1] not in outsideCoords:
                    #Mark as outside point
                    pipes[coords[0]][coords[1] + 1] = "O"
                    #Add coords to outsideCoords
                    outsideCoords.append([coords[0], coords[1] + 1])
            
    return pipes

def isValidCoordinate(pipes, xCoordinate, yCoordinate):
    """Returns False if x or y coordinate is out of bounds otherwise return True"""
    if xCoordinate < 0 or xCoordinate > len(pipes)-1:
        return False
    if yCoordinate < 0 or yCoordinate > len(pipes[0])-1:
        return False
    return True

def printPipes(pipes):
    """Prints pipes, pretty much just used for testing purposes"""
    for i in range(len(pipes)):
        for j in range(len(pipes[i])):
            print(pipes[i][j], end="")
        print()

#Formatting
pipes = getPipes()
pipes = pipes.split("\n")
pipes = [list(x) for x in pipes]

#Find 'S' as the starting location:
for i, row in enumerate(pipes):
    for j, char in enumerate(row):
        if char == "S":
            sCoordinate = [i, j] #Keep track of the start Coordinate
            currentCoords = [i, j] #Keeps track of current coordinates
            break

#Find the next move based on the location of the start ("S")
#Check for valid path up (x coordinate - 1)
if pipes[currentCoords[0] - 1][currentCoords[1]] in ['|', 'F', '7']:
    currentCoords[0] -= 1
    previousDirection = "Up"
#Check for valid path down (x coordinate - 1) if there is no valid path up
elif pipes[currentCoords[0] + 1][currentCoords[1]] in ['|', 'L', 'J']:
    currentCoords[0] += 1
    previousDirection = "Down"
#The start has to be connected by exactly two sides, so if it's not up or down, it has to be left and right
else:
    pipes[1] -= 1
    previousDirection = "Left"
    
pathCoords = [[currentCoords[0], currentCoords[1]]]

#Loop through until we get back to "S"
while pipes[currentCoords[0]][currentCoords[1]] != "S":
    
    #Keep track of the current character
    currentChar = pipes[currentCoords[0]][currentCoords[1]]
    
    if currentChar == "|":
        #If the previous direction was up, that means it came from the bottom, so we need to move up
        if previousDirection == "Up":
            currentCoords[0] -= 1
            previousDirection = "Up"
        #This means if came from above, so we have to move down
        else:
            currentCoords[0] += 1
            previousDirection = "Down"
            
    elif currentChar == "-":
        #If we came from the left, that means we continue to move left
        if previousDirection == "Left":
            currentCoords[1] -= 1
            previousDirection = "Left"
        #This means if came from the right, so we have to keep moving to the right
        else:
            currentCoords[1] += 1
            previousDirection = "Right"
            
    elif currentChar == "L":
        #If coming from the left, move up
        if previousDirection == "Left":
            currentCoords[0] -= 1
            previousDirection = "Up"
        #If coming from up, move to the right
        else:
            currentCoords[1] += 1
            previousDirection = "Right"
            
    elif currentChar == "J":
        #If coming from right, move up
        if previousDirection == "Right":
            currentCoords[0] -= 1
            previousDirection = "Up"
        #If coming from up move to the left
        else:
            currentCoords[1] -= 1
            previousDirection = "Left"
            
    elif currentChar == "7":
        #If coming from the bottom, move left
        if previousDirection == "Up":
            currentCoords[1] -= 1
            previousDirection = "Left"
        #If coming from the right, move down
        else:
            currentCoords[0] += 1
            previousDirection = "Down"
            
    elif currentChar == "F":
        #If coming from the bottom, move right
        if previousDirection == "Up":
            currentCoords[1] += 1
            previousDirection = "Right"
        #If coming from right, move down
        else:
            currentCoords[0] += 1
            previousDirection = "Down"
    
    pathCoords.append([currentCoords[0], currentCoords[1]])
        
#Replace non path characters with "I"
for i, row in enumerate(pipes):
    for j in range(len(row)):
        if [i,j] not in pathCoords:
            pipes[i][j] = "I"
            
#Replace S with correct character
#This assumes S has to be not a '|' or a '-'
connectedBottom = pipes[sCoordinate[0] + 1][sCoordinate[1]] in ['|', 'L', 'J']
connectedTop = pipes[sCoordinate[0] - 1][sCoordinate[1]] in ['|', 'F', '7']
connectedRight = pipes[sCoordinate[0]][sCoordinate[1] + 1] in ['-', 'J', '7']
connectedLeft = pipes[sCoordinate[0]][sCoordinate[1] - 1] in ['-', 'F', 'L']
#Checks for F
if connectedRight and connectedBottom:
    pipes[sCoordinate[0]][sCoordinate[1]] = "F"
#Checks for L
if connectedRight and connectedTop:
    pipes[sCoordinate[0]][sCoordinate[1]] = "L"
#Checks for 7
if connectedLeft and connectedBottom:
    pipes[sCoordinate[0]][sCoordinate[1]] = "7"
#Checks for J
if connectedLeft and connectedTop:
    pipes[sCoordinate[0]][sCoordinate[1]] = "J"

    
#Put empty spaces between every row and column
spacedPipes = spacePipes(pipes)
#Fills the map with outside points
spacedPipes = fillBoard(spacedPipes, 0, 0)

#Number of points inside the the loop
numInsidePoints = 0
for row in spacedPipes:
    numInsidePoints += row.count("I")

print(f"Number of points inside the loop: {numInsidePoints}")

Number of points inside the loop: 595


# Notes:

In [9]:
pipes = """..........
.S------7.
.|F----7|.
.||....||.
.||....||.
.|L-7F-J|.
.|..||..|.
.L--JL--J.
.........."""
pipes = pipes.split("\n")
pipes = [list(x) for x in pipes]

print("Turns this:")
#Print board
for i in range(len(pipes)):
    for j in range(len(pipes[i])):
        print(pipes[i][j], end="")
    print()

spacedPipes = spacePipes(pipes)

print("\n\nInto this:")
#Print board
for i in range(len(spacedPipes)):
    for j in range(len(spacedPipes[i])):
        print(spacedPipes[i][j], end="")
    print()

Turns this:
..........
.S------7.
.|F----7|.
.||....||.
.||....||.
.|L-7F-J|.
.|..||..|.
.L--JL--J.
..........


Into this:
.#.#.#.#.#.#.#.#.#.
###################
.#S#------------7#.
################|##
.#|#F---------7#|#.
##|#|#########|#|##
.#|#|#.#.#.#.#|#|#.
##|#|#########|#|##
.#|#|#.#.#.#.#|#|#.
##|#|#########|#|##
.#|#L---7#F---J#|#.
##|#####|#|#####|##
.#|#.#.#|#|#.#.#|#.
##|#####|#|#####|##
.#L-----J#L-----J#.
###################
.#.#.#.#.#.#.#.#.#.


In [13]:
"""This flood fill method works great, except for that on too big of an input (like my input) it recurses too deep, and
the kernel dies. It's unclear as to why the kernel dies rather than getting a recursion depth error, but either way, it
doesn't work"""

def traverseBoard(pipes, xCoordinate, yCoordinate, outsideCoords=[]):
    #Mark current point as an outside point
    outsideCoords.append([xCoordinate, yCoordinate])
    
    #Checks to make sure the coordinate above the current coordinate is in bounds
    if isValidCoordinate(pipes, xCoordinate - 1, yCoordinate):
        #If the pipe is marked as either a filler space, or an inside point, call this function on that point
        if pipes[xCoordinate - 1][yCoordinate] in ["#", "I"] and [xCoordinate - 1, yCoordinate] not in outsideCoords:
            outsideCoords = traverseBoard(pipes, xCoordinate - 1, yCoordinate, outsideCoords)
    
    #Checks to make sure the coordinate below the current coordinate is in bounds
    if isValidCoordinate(pipes, xCoordinate + 1, yCoordinate):
        #If the pipe is marked as either a filler space, or an inside point, call this function on that point
        if pipes[xCoordinate + 1][yCoordinate] in ["#", "I"] and [xCoordinate + 1, yCoordinate] not in outsideCoords:
            outsideCoords = traverseBoard(pipes, xCoordinate + 1, yCoordinate, outsideCoords)
    
    #Checks to make sure the coordinate to the left of the current coordinate is in bounds
    if isValidCoordinate(pipes, xCoordinate, yCoordinate - 1):
        #If the pipe is marked as either a filler space, or an inside point, call this function on that point
        if pipes[xCoordinate][yCoordinate - 1] in ["#", "I"]  and [xCoordinate, yCoordinate - 1] not in outsideCoords:
            outsideCoords = traverseBoard(pipes, xCoordinate, yCoordinate - 1, outsideCoords)
    
    #Checks to make sure the coordinate to the right of the current coordinate is in bounds
    if isValidCoordinate(pipes, xCoordinate, yCoordinate + 1):
        #If the pipe is marked as either a filler space, or an inside point, call this function on that point
        if pipes[xCoordinate][yCoordinate + 1] in ["#", "I"] and [xCoordinate, yCoordinate + 1] not in outsideCoords:
            outsideCoords = traverseBoard(pipes, xCoordinate, yCoordinate + 1, outsideCoords)
            
    return outsideCoords